## Critical path showcase

__UNDER CONSTRUCTION__

This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the barges done, another (third) vessel moves the entire amount from site 1 to site 2.
* five barges move an amount from site 0 to site 1. When the barges done,another (sixth) vessel moves the entire amount from site 1 to site 2.


#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, AlteredStepEnv
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities

#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 to site 2 whereafter a larger vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 2 * total_amount,
        "level": 2 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site,
                    "state": "full",
                    "id_": "default_reservations",
                }
            ],
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 100
    activities["activity_vessel0"] = model.SequentialActivity(
        env=my_env,
        name="sequential_v0",
        registry=registry,
        sub_processes=[
            model.BasicActivity(
                env=my_env,
                name="basic activity vessel_last",
                registry=registry,
                duration=duration,
                additional_logs=[vessel_last],
                start_event=[
                    {"type": "container", "concept": to_site, "state": "full"}
                ],
            ),
            model.MoveActivity(
                env=my_env,
                name="sailing empty: vessel_last",
                registry=registry,
                mover=vessel_last,
                destination=from_site,
                duration=duration,
            ),
            model.ShiftAmountActivity(
                env=my_env,
                name="loading vessel_last",
                registry=registry,
                processor=vessel_last,
                origin=from_site,
                destination=vessel_last,
                amount=amount,
                duration=500 * amount,
                requested_resources=requested_resources,
            ),
            model.MoveActivity(
                env=my_env,
                name="sailing full vessel_last",
                registry=registry,
                mover=vessel_last,
                destination=to_site2,
                duration=duration,
            ),
            model.ShiftAmountActivity(
                env=my_env,
                name="unloading vessel_last",
                registry=registry,
                processor=vessel_last,
                origin=vessel_last,
                destination=to_site2,
                amount=amount,
                duration=duration,
                requested_resources=requested_resources,
            ),
        ],
    )

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 3. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [3]:
simulation_data_out = demo_data(2, 100, env=AlteredStepEnv)


In [4]:
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,d5a008af-196e-42d6-a6c3-5973c3a5530f,basic activity vessel_last,vessel_last,1970-01-01 00:00:00,1970-01-02 05:51:40,1 days 05:51:40,WAITING,6daefa48-b885-4664-a573-d1a9a2544761,False
1,b7f81e27-45f5-4fdc-8153-f2ec7afe8261,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,96677819-1854-4b86-a92c-6f35329ab199,True
2,2eb02f98-2162-4c28-ace3-bc4f1a65ff6c,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,ff8fba25-d149-411b-a030-3601fdda19bf,False
3,ca3b1c0e-0824-473f-87a3-656723ef4475,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,91c8ee87-40bf-4851-8040-028643210c9e,True
4,7868bd9c-4c8c-444e-b91a-5dd64f52a142,sailing empty:barge_1,barge_1,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,ee2e36c9-f5ee-43fa-a64f-c82d91915bd2,False


In [5]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=critical_df,
                     id_map=simulation_data_out['activity_list'])
                     

#### 4. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [11]:
# run simulation
simulation_data_out = demo_data(2, 100)

# get critical path as recorded_activities_dataframe with column 'is_critical'
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,eb62ac3f-a7dc-4eda-932c-87d8aa0fa9fa,basic activity vessel_last,vessel_last,1970-01-01 00:00:00,1970-01-02 05:51:40,1 days 05:51:40,WAITING,e762f43d-b174-4468-a22e-3572820805a0,True
1,1433e28e-5c00-4830-b047-8cca100893e5,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,594124d0-b7e9-424d-ab48-c25d4d33aceb,True
2,a8943740-e492-4a38-a073-2f7fe5c7d064,basic activity:barge_1,barge_1,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,03f236e8-357d-4c2b-865c-9e05a97dc874,False
3,b39406b1-5a7f-467c-8a64-dd0a16257548,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,e2170972-bb27-4a60-b911-b5a39f49c731,True
4,23d963f9-3ced-478e-858d-2c474512e24d,sailing empty:barge_1,barge_1,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,895444d3-7a08-47f7-ab83-bbfdd0d2e215,False


In [12]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=critical_df,
                     id_map=simulation_data_out['activity_list'])

#### 5. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 5 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [8]:
# run simulation
simulation_data_out = demo_data(5, 100, env=AlteredStepEnv)

# get critical path as recorded_activities_dataframe with column 'is_critical'
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
0,356b4843-b744-4e68-b157-734107e45079,basic activity vessel_last,vessel_last,1970-01-01,1970-01-01 16:06:40,0 days 16:06:40,WAITING,4c00e2bf-7511-4cde-85b1-eda4ebeafaa8,False
1,a5cff019-df63-4b75-bd61-8b7d42781696,basic activity:barge_0,barge_0,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,98290d3c-ff34-4ca7-aceb-da9d23c8f513,True
2,68637874-c992-401a-9cd0-6b645f2ae0b0,basic activity:barge_1,barge_1,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,8f2e53c5-571a-4515-b408-31b9a33fa246,False
3,2790874e-e987-4ea1-9029-39b13616b290,basic activity:barge_2,barge_2,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,2a661b60-e0cd-4581-912a-2e6c4500573a,False
4,8cc2fdc3-0e47-41fb-8c89-8e7c16787137,basic activity:barge_3,barge_3,1970-01-01,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,88a74a23-9902-4838-9d50-a146d607aed9,False


In [9]:
plot.get_gantt_chart(simulation_data_out['object_list'],
                     critical_path_dataframe=critical_df,
                     id_map=simulation_data_out['activity_list'])